In [51]:
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import wittgenstein as lw  # RIPPER
from Orange.classification import CN2Learner 

In [52]:
df = pd.read_csv("/home/adel/Documents/Code/Ant-Miner/datasets/hepatitis.csv", dtype=str)
df

,age,sex,steroid,antivirals,fatigue,malaise,anorexia,liver_big,liver_firm,spleen_palpable,spiders,ascites,varices,bilirubin,alk_phosphate,sgot,albumin,protime,histology,class
0,"[30, 40)",male,False,False,False,False,False,False,False,False,False,False,False,unknown,"[80, 90)","[10, 20)",unknown,unknown,False,neg
1,"[50, 60)",female,False,False,True,False,False,False,False,False,False,False,False,unknown,"[130, 140)","[40, 50)",unknown,unknown,False,neg
2,unknown,female,True,False,True,False,False,True,False,False,False,False,False,unknown,"[90, 100)","[30, 40)",unknown,unknown,False,neg
3,"[30, 40)",female,unknown,True,False,False,False,True,False,False,False,False,False,unknown,"[40, 50)","[50, 60)",unknown,"[80, 90)",False,neg
4,"[30, 40)",female,True,False,False,False,False,True,False,False,False,False,False,unknown,unknown,"[200, 210)",unknown,unknown,False,neg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,"[40, 50)",female,True,False,True,True,True,True,False,False,True,True,True,unknown,unknown,"[240, 250)",unknown,"[50, 60)",True,pos
151,"[40, 50)",female,True,False,True,False,False,True,True,False,False,False,False,unknown,"[120, 130)","[140, 150)",unknown,unknown,True,neg
152,"[60, 70)",female,False,False,True,True,False,False,True,False,True,False,False,unknown,"[70, 80)","[20, 30)",unknown,unknown,True,neg
153,"[50, 60)",male,False,False,True,False,False,True,False,True,True,False,True,unknown,"[80, 90)","[10, 20)",unknown,"[40, 50)",True,neg


In [53]:
X = df.drop("class", axis=1)
y = df["class"]

In [54]:
le_y = LabelEncoder()
y_enc = le_y.fit_transform(y)

In [55]:
scoring = {
    "accuracy": make_scorer(accuracy_score),
    "precision": make_scorer(precision_score, average="macro", zero_division=0),
    "recall": make_scorer(recall_score, average="macro", zero_division=0),
    "f1": make_scorer(f1_score, average="macro", zero_division=0),
}
results = []

In [56]:
# One-hot encoding for sklearn models
categorical_features = X.columns.tolist()
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_features)]
)

# Models (non-rule-based)
models = {
    "C4.5 (DecisionTree)": DecisionTreeClassifier(criterion="entropy", random_state=42),
    "SVM": SVC(kernel="rbf", random_state=42),
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "NaiveBayes": GaussianNB(),
    "NeuralNetwork": MLPClassifier(hidden_layer_sizes=(64,), max_iter=500, random_state=42),
}

In [57]:
# === Sklearn models ===
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, clf in models.items():
    model = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", clf)])
    scores = cross_validate(model, X, y_enc, cv=cv, scoring=scoring)
    # averager results with std deviation
    row = {
        "Model": name,
        "Accuracy": f"{np.mean(scores['test_accuracy']) * 100:.2f} ± {np.std(scores['test_accuracy']) * 100:.2f}",
        "F1-score": f"{np.mean(scores['test_f1']) * 100:.2f} ± {np.std(scores['test_f1']) * 100:.2f}",
        "Recall": f"{np.mean(scores['test_recall']) * 100:.2f} ± {np.std(scores['test_recall']) * 100:.2f}",
        "Precision": f"{np.mean(scores['test_precision']) * 100:.2f} ± {np.std(scores['test_precision']) * 100:.2f}",
        "Nb of Rules": np.nan,
        "Term/Rule Ratio": np.nan,
    }
    results.append(row)

In [58]:
# === RIPPER (rule-based) ===
X_le = X.copy()
for col in X_le.columns:
    le = LabelEncoder()
    X_le[col] = le.fit_transform(X_le[col])

ripper = lw.RIPPER(random_state=42)

accs, precs, recs, f1s, n_rules, tr_ratios = [], [], [], [], [], []
for train_idx, test_idx in cv.split(X_le, y_enc):
    X_train, X_test = X_le.iloc[train_idx], X_le.iloc[test_idx]
    y_train, y_test = y_enc[train_idx], y_enc[test_idx]
    ripper.fit(X_train, y_train, pos_class=1)
    y_pred = ripper.predict(X_test)

    accs.append(accuracy_score(y_test, y_pred))
    precs.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
    recs.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
    f1s.append(f1_score(y_test, y_pred, average='macro', zero_division=0))

    # extract rules
    rules = ripper.ruleset_.rules
    n_rules.append(len(rules))
    if len(rules) > 0:
        tr_ratios.append(np.mean([len(rule.conds) for rule in rules]))
    else:
        tr_ratios.append(0)

results.append({
    "Model": "RIPPER",
    "Accuracy": f"{np.mean(accs) * 100:.2f} ± {np.std(accs) * 100:.2f}",
    "F1-score": f"{np.mean(f1s) * 100:.2f} ± {np.std(f1s) * 100:.2f}",
    "Recall": f"{np.mean(recs) * 100:.2f} ± {np.std(recs) * 100:.2f}",
    "Precision": f"{np.mean(precs) * 100:.2f} ± {np.std(precs) * 100:.2f}",
    "Nb of Rules": f"{np.mean(n_rules):.2f} ± {np.std(n_rules):.2f}",
    "Term/Rule Ratio": f"{np.mean(tr_ratios):.2f} ± {np.std(tr_ratios):.2f}",
})

In [59]:
# ====================================
# CN2 (rule-based, Orange3)
# ====================================
# Convert to Orange Table
from Orange.data import Table, Domain, DiscreteVariable

domain = Domain(
    [DiscreteVariable.make(name) for name in X_le.columns],
    DiscreteVariable.make("target", values=[str(c) for c in np.unique(y_enc)])
)
data = Table(domain, np.hstack([X_le.values, y_enc.reshape(-1, 1)]))

cn2 = CN2Learner()

accs, precs, recs, f1s, n_rules, tr_ratios = [], [], [], [], [], []
for train_idx, test_idx in cv.split(X_le, y_enc):
    train_data = data[train_idx]
    test_data = data[test_idx]

    cn2_classifier = cn2(train_data)
    y_pred = np.array([int(str(c)) for c in cn2_classifier(test_data)])

    accs.append(accuracy_score(y_enc[test_idx], y_pred))
    precs.append(precision_score(y_enc[test_idx], y_pred, average='macro', zero_division=0))
    recs.append(recall_score(y_enc[test_idx], y_pred, average='macro', zero_division=0))
    f1s.append(f1_score(y_enc[test_idx], y_pred, average='macro', zero_division=0))

    rules = cn2_classifier.rule_list
    n_rules.append(len(rules))
    if len(rules) > 0:
        tr_ratios.append(np.mean([len(r.selectors) for r in rules]))
    else:
        tr_ratios.append(0)

results.append({
    "Model": "CN2",
    "Accuracy": f"{np.mean(accs) * 100:.2f} ± {np.std(accs) * 100:.2f}",
    "F1-score": f"{np.mean(f1s) * 100:.2f} ± {np.std(f1s) * 100:.2f}",
    "Recall": f"{np.mean(recs) * 100:.2f} ± {np.std(recs) * 100:.2f}",
    "Precision": f"{np.mean(precs) * 100:.2f} ± {np.std(precs) * 100:.2f}",
    "Nb of Rules": f"{np.mean(n_rules):.2f} ± {np.std(n_rules):.2f}",
    "Term/Rule Ratio": f"{np.mean(tr_ratios):.2f} ± {np.std(tr_ratios):.2f}",
})

In [60]:
df_results = pd.DataFrame(results)
df_results

,Model,Accuracy,F1-score,Recall,Precision,Nb of Rules,Term/Rule Ratio
0,C4.5 (DecisionTree),78.71 ± 5.62,69.60 ± 7.94,71.85 ± 9.62,68.92 ± 7.62,NaN,NaN
1,SVM,83.87 ± 3.53,68.13 ± 5.48,65.49 ± 4.70,85.02 ± 9.84,NaN,NaN
2,RandomForest,85.81 ± 6.32,74.12 ± 9.49,70.73 ± 8.39,85.44 ± 12.91,NaN,NaN
3,NaiveBayes,57.42 ± 10.68,50.95 ± 11.00,55.16 ± 14.06,53.33 ± 9.21,NaN,NaN
4,NeuralNetwork,81.29 ± 4.74,67.31 ± 11.61,67.44 ± 11.66,69.67 ± 9.61,NaN,NaN
5,RIPPER,80.65 ± 2.89,70.13 ± 3.49,70.55 ± 5.15,71.35 ± 3.48,3.00 ± 0.63,1.87 ± 0.34
6,CN2,70.97 ± 10.40,59.49 ± 12.06,59.89 ± 12.70,59.59 ± 11.11,41.20 ± 6.91,0.97 ± 0.00
